<center>
<h1>Detección y conteo de vulgaridades INCLUYENDO y EXCLUYENDO palabras con spanlp</h1><br>
</center>
<img src="data/static/imagen.png" alt="spanlp">

**Problema:**
<br><br>
Los tweets recolectados (data/tweets.csv') contienen palabras y frases que son considerados vulgaridades pero el cliente quiere que no sean detectadas como tal. Ej: “ladrones” “porqueria” “hdp”  (_Hay que ignorarlas_)

De otro lado queremos que palabras como: “pésima” “peor” “robo” “no sirve” “mala” “malo” (_Hay que tenerlas en cuenta_)
<br><br>
**Objetivos:**
<br>
* Leer el archivo de datos que contiene los tweets.
* Detectar e indicar si los datos tienen vulgaridades.
* Contar la cantidad de vulgaridades que hay.
* Pedir al usuario las palabras que quiere incluir o excluir en la busqueda

In [1]:
from spanlp.palabrota import Palabrota
from spanlp.domain.countries import Country
import csv

In [2]:
def leer_archivo(nombre_archivo:str='', separador_datos:str='', columna:str=''):
    tweets = []
    with open(nombre_archivo, encoding='utf-8') as data:
        reader = csv.DictReader(data, delimiter=separador_datos)
        for fila in reader:
            tweets.append(fila[columna])
    return tweets

In [15]:
def validar_vulgaridades(data:list=None, pais:list=None, incluir:list=None, excluir:list=None):
    if data is None or len(data) == 0:
        raise Exception("Oiga! para validar vulgaridades debe enviar datos. esta enviando datos nulos o vacios.")
    
    contador_palabrotas = 0
    validador = Palabrota(countries=pais, include=incluir, exclude=excluir)
    
    for texto in data:
        if isinstance(texto, str):
            es_palabrota = validador.contains_palabrota(texto)
            if es_palabrota:
                contador_palabrotas += 1
    return (contador_palabrotas>0,contador_palabrotas)

In [4]:
comments = leer_archivo(nombre_archivo='data/tweets.csv', separador_datos='|', columna='comment_1')
contiene, cantidad = validar_vulgaridades(comments)

print(f"Los datos contienen {cantidad} vulgaridades." if contiene else "Los datos no contienen vulgaridades.")

Los datos contienen 7 vulgaridades.


**Entender los parámetros de Palabrota y como funciona Country**

In [5]:
print(Country.COLOMBIA)
#help(Palabrota)

Country.COLOMBIA


**Detectando palabrotas por paises**

In [18]:
frases = ["carajo perdi el examen", "me presta la cuchara?", "pendejo gané el examen", "ese forro está bonito", 
          "si por favor, si quiero un pitillo"]

validador_CO = Palabrota(countries=[Country.COLOMBIA], include=["carajo"])
validador_VE = Palabrota(countries=[Country.VENEZUELA])
validador_AR = Palabrota(countries=[Country.ARGENTINA])
validador_ME = Palabrota(countries=[Country.MEXICO])

for frase in frases:
    contiene_CO = validador_CO.contains_palabrota(frase)
    contiene_VE = validador_VE.contains_palabrota(frase)
    contiene_AR = validador_AR.contains_palabrota(frase)
    contiene_ME = validador_ME.contains_palabrota(frase)
    
    print(f"Frase: '{frase}'")
    print(f" CO: {contiene_CO} VE: {contiene_VE} AR: {contiene_AR} ME: {contiene_ME}")
    print()

Frase: 'carajo perdi el examen'
 CO: True VE: True AR: False ME: False

Frase: 'me presta la cuchara?'
 CO: False VE: True AR: False ME: False

Frase: 'pendejo gané el examen'
 CO: True VE: True AR: False ME: True

Frase: 'ese forro está bonito'
 CO: False VE: False AR: True ME: False

Frase: 'si por favor, si quiero un pitillo'
 CO: False VE: False AR: False ME: True



**Recibir el pais dinámicamente del usuario y leer tweets**

In [7]:
def pedir_pais():
    print(""" Paises disponibles para el análisis: 
            1 Colombia
            2 Venezuela
            3 Mexico
            4 Argentina
            5 Todos""")
    pais_seleccionado = int(input("Ingrese:"))
    return {
        1: [Country.COLOMBIA],
        2: [Country.VENEZUELA],
        3: [Country.MEXICO],
        4: [Country.ARGENTINA],
        5: [Country.COLOMBIA, Country.VENEZUELA, Country.MEXICO, Country.ARGENTINA]
    }.get(pais_seleccionado, None)

In [10]:
def pedir_palabras_incluir_excluir():
    print("¿Desea incluir o excluir palabras en la busqueda? (SI/NO):")
    quiere_incluir_excluir = input("Respuesta (SI/NO) :")
    if len(quiere_incluir_excluir) > 0 and quiere_incluir_excluir.lower() == "si":
        print("Ingrese la palabras a INCLUIR (separadas por coma (,)) :")
        palabras_a_incluir = input("Incluir:")
        print("Ingrese la palabras a EXCLUIR (separadas por coma (,)) :")
        palabras_a_excluir = input("Excluir:")
        return (palabras_a_incluir.split(","), palabras_a_excluir.split(","))
    else:
        return (None,None)

In [20]:
comments = leer_archivo(nombre_archivo='data/tweets.csv', separador_datos='|', columna='comment_1')
print(comments)
pais = pedir_pais()
palabras_a_incluir, palabras_a_excluir = pedir_palabras_incluir_excluir()
contiene, cantidad = validar_vulgaridades(comments, pais, palabras_a_incluir, palabras_a_excluir)

print(f"Los datos contienen {cantidad} vulgaridades para {pais}." if contiene else "Los datos no contienen vulgaridades.")

['No tengo servicio !? Que es lo que pasa? ladrones', 'Soluciones más claras a los usuarios :( el servicio es una porqueria', 'Estoy sin servicio de datos, de voz, el servicio hogar es más lento de lo normal, abro la app y está para lo único que funciona es para pagar,de resto no sirve nada, que pasa', 'Buenas tardes, nos pueden indicar por que desde hace varios días el servicio de telefonía y de datos en la ciudad es de mala calidad, está una mierda.', 'Las llamadas presentan mucha interferencia y la conexión a redes sociales o Internet desde el celular es mala.', 'Trabajan todos los días para mirar como robar a los usuarios, no me crean imbecil', 'Pero que pesima señal esta semana q paso y hoy va por lo mismo que pasa den una respuesta malditos....', '@operador @operadorTeAyuda tienen problemas con el servicio?? El Internet está una mierda!!', 'Y si en vez de habilitar medios de pago, solucionan el problema que tienen de red? Nos están dejando incomunicados! HDP', 'no sea pendejo el 

In [ ]:
Ingrese:5
¿Desea incluir o excluir palabras en la busqueda? (SI/NO):
Respuesta (SI/NO) :si
Ingrese la palabras a INCLUIR (separadas por coma (,)) :
Incluir:peor,sistema,datos,mirar
Ingrese la palabras a EXCLUIR (separadas por coma (,)) :
Excluir:ladrones,porqueria,mierda,pendejo,imbecil
Los datos contienen 5 vulgaridades para 
[<Country.COLOMBIA: 'COL'>, <Country.VENEZUELA: 'VEN'>, <Country.MEXICO: 'MEX'>, <Country.ARGENTINA: 'ARG'>].

**RETO**

Modificar la nueva funcion pedir_palabras_incluir_excluir() para que:
* Valide si el usuario no ingresa nada al pedir las palabras.
* Valide si el usuario ingresa las palabras separado por otro caracter distinto de coma (,)
* solicite al usuario por separado si quiere o no incluir o excluir palabras.